In [1]:
%load_ext autoreload
%autoreload 2
from Methods import sentenceSimilarityMethods,JobOffers,dataframeHelpers

In [2]:
import json
import os
import nltk
import re
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
from scipy import spatial

In [4]:
model = hub.load("/Users/klong/Desktop/Thesis/universal-sentence-encoder_4/")
def embed(input):
  return model(input)

In [5]:
autoreload JobOffers,dataframeHelpers

In [6]:
# Only run this if you want to redo the translations
#JobOffers.writeTranslatedJobs(100)

In [7]:
slimmedActsDf = dataframeHelpers.buildActivitiesDFV26()

In [8]:
slimmedActsDf

,ACTIVITY_CONTENT,ACTIVITY_ID
0,Where and how are the information and data gai...,2
1,"Observing, receiving, and otherwise obtaining...",3
2,Research geological features or processes.,4
3,Conduct climatological research.,5
4,Investigate the environmental impact of indust...,6
...,...,...
283,Providing guidance and expert advice to manage...,290
284,Performing day-to-day administrative tasks suc...,291
285,Obtain permits.,292
286,"Recruiting, interviewing, selecting, hiring, ...",293


In [9]:
#got stuck here
jobsDf = dataframeHelpers.buildJobsDF()

In [10]:
jobsDf

,JOB_ID,ISCO,JOB_CONTENT,ORIGINAL_CONTENT
0,12220025,21420,[We are a successful and independent engineeri...,<p>Wir sind ein erfolgreiches und unabhängiges...
1,10814567,21420,[We are a renowned and successful engineering ...,Wir sind eine renommierte und erfolgreiche Ing...
2,12401674,21420,[Bill Weyermann Partner AG has been providing ...,Bill Weyermann Partner AG erbringt seit über\n...
3,12473085,62100,[Administration communale de ValdeRuzForstwart...,Administration communale de Val-de-RuzForstwar...
4,12905109,26220;24210,[The Straumann Group is internationally succes...,Le groupe Straumann connaît un succès internat...
...,...,...,...,...
95,13469747,62100,[Commune de Lausanne Service des parcs et doma...,Commune de Lausanne - Service des parcs et dom...
96,13494906,24210,[Quality Assurance Engineer Bern Switzerland Y...,"Quality Assurance Engineer\n\n\nBern, Switzerl..."
97,13517245,21420;31180,[Solution-oriented creative and dynamic We are...,"Lösungsorientiert, kreativ und dynamisch: Wir ..."
98,13461980,21620,[Green BlueGarden garden and landscaping EFZLa...,Grün & BlauGärtner/in (Garten- und Landschafts...


In [11]:
embeddings = dataframeHelpers.getEmbeddings(slimmedActsDf, jobsDf)

In [12]:
slimmedActsDf[slimmedActsDf["ACTIVITY_CONTENT"].isna()]


,ACTIVITY_CONTENT,ACTIVITY_ID


In [13]:
jobsSentences = np.array([y for x in jobsDf["JOB_CONTENT"] for y in x])

In [14]:
jobsDf = dataframeHelpers.addEmbeddingsToJobs(jobsDf, embeddings)
slimmedActsDf = dataframeHelpers.addEmbeddingsToActivities(slimmedActsDf, embeddings)

In [15]:
resultsDf = pd.DataFrame()

In [16]:
for label in ["JOB_ID", "JOB_SCORES"]:
    resultsDf[label] = jobsDf[label]

In [17]:
resultsDf = resultsDf.merge(slimmedActsDf, how = "cross")

In [18]:
resultsDf = resultsDf[["JOB_ID", "ACTIVITY_ID", "JOB_SCORES", "ACTIVITY_SCORES"]]

In [19]:
def compareSentences(row):
    for sentence in resultsDf["JOB_SCORES"]:
        return spatial.distance.cosine(row["JOB_SCORES"][sentence], row["ACTIVITY_SCORES"])

In [20]:
def distCalc(row):
    distArray = []
    for entry in row["JOB_SCORES"] :
        dist = spatial.distance.cosine(entry, row["ACTIVITY_SCORES"])
        distArray.append(dist)
    return distArray
    

In [21]:
resultsDf["DISTANCES"] = resultsDf.apply(distCalc, axis =1)

In [22]:
def minDistanceFinder(row) : 
    minDistance = min(row["DISTANCES"])
    return minDistance

In [23]:
def minDistanceSentence(row, jobs) : 
    minDistanceIdx = np.argmin(row["DISTANCES"])
    df = jobs[jobs["JOB_ID"] == row["JOB_ID"]]
    if df.empty:
        return ""
    content = df.iloc[0]["JOB_CONTENT"]
    return content[minDistanceIdx]

In [24]:
resultsDf["LOWEST_DISTANCE"] = resultsDf.apply(minDistanceFinder, axis =1)
resultsDf["LOWEST_DISTANCE_SENTENCE"] = resultsDf.apply(minDistanceSentence, args=[jobsDf], axis =1)


In [25]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES,DISTANCES,LOWEST_DISTANCE,LOWEST_DISTANCE_SENTENCE
0,12220025,2,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.027285306, shape=(), dtype=float3...","[0.9333634227514267, 0.9425690993666649, 0.858...",0.729741,They want to gain experience in project plann...
1,12220025,3,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.07172423, shape=(), dtype=float32...","[1.106818176805973, 0.9665663540363312, 0.9541...",0.721484,Further information and application We are ha...
2,12220025,4,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.045720328, shape=(), dtype=float3...","[1.0460460484027863, 0.9370203092694283, 0.940...",0.807079,Haldi Design AG Schönried
3,12220025,5,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.05692438, shape=(), dtype=float32...","[1.048239592462778, 0.9645586386322975, 0.8781...",0.866480,Haldi Design AG Schönried
4,12220025,6,"((tf.Tensor(-0.037420016, shape=(), dtype=floa...","(tf.Tensor(0.07220104, shape=(), dtype=float32...","[1.0029801961500198, 0.8679462969303131, 0.904...",0.837572,They want to gain experience in project plann...
...,...,...,...,...,...,...,...
28795,13462067,290,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.027627837, shape=(), dtype=float3...","[0.9878562754020095, 1.0708704590797424, 1.072...",0.987856,Haas Gartenbau AGGarden and Landscaping EFZBer...
28796,13462067,291,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(-0.008161316, shape=(), dtype=float...","[0.9724013563245535, 1.0722142532467842, 1.026...",0.972401,Haas Gartenbau AGGarden and Landscaping EFZBer...
28797,13462067,292,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.019920168, shape=(), dtype=float3...","[0.9978231620043516, 0.9633813798427582, 0.986...",0.962735,9 Updated19
28798,13462067,293,"((tf.Tensor(-0.0019656876, shape=(), dtype=flo...","(tf.Tensor(0.0046779006, shape=(), dtype=float...","[0.9451519772410393, 1.0347175188362598, 0.986...",0.880178,9 Updated19


In [26]:
def bestMatchReturn(row, results, actsDf, match, column_name) :
    bestMatch = results[results["JOB_ID"] == row["JOB_ID"]]
    minEntry = bestMatch[match].idxmin()
    rowMatch = bestMatch.loc[minEntry,:]  
    #return actsDf[actsDf["ACTIVITY_ID"] == rowMatch["ACTIVITY_ID"]]["ACTIVITY_ID"]
    content = actsDf[actsDf["ACTIVITY_ID"] == rowMatch["ACTIVITY_ID"]]                  
    return content[column_name].iloc[0]            


In [27]:
def bestNMatchReturn(row, results, actsDf, match, column_name, num) :
    jobMatch = results[results["JOB_ID"] == row["JOB_ID"]]
    minEntries = jobMatch[match].argsort()[:num]
    rowMatch = jobMatch.iloc[minEntries,:]
    content = [actsDf[actsDf["ACTIVITY_ID"] == x][column_name].iloc[0] for x in rowMatch["ACTIVITY_ID"]]
    return content

In [28]:
def averageNSentences(row, n):
    return np.average(np.sort(row["DISTANCES"])[:n])

In [29]:
resultsDf["AVERAGE_DISTANCE_3SENT"] = resultsDf.apply(averageNSentences, args=[3], axis=1)

In [30]:
row = resultsDf.loc[2]
row

JOB_ID                                                               12220025
ACTIVITY_ID                                                                 4
JOB_SCORES                  ((tf.Tensor(-0.037420016, shape=(), dtype=floa...
ACTIVITY_SCORES             (tf.Tensor(0.045720328, shape=(), dtype=float3...
DISTANCES                   [1.0460460484027863, 0.9370203092694283, 0.940...
LOWEST_DISTANCE                                                      0.807079
LOWEST_DISTANCE_SENTENCE                            Haldi Design AG Schönried
AVERAGE_DISTANCE_3SENT                                                0.82608
Name: 2, dtype: object

In [31]:
slimmedActsDf[slimmedActsDf["ACTIVITY_ID"] == row["ACTIVITY_ID"]]

,ACTIVITY_CONTENT,ACTIVITY_ID,ACTIVITY_SCORES
2,Research geological features or processes.,4,"(tf.Tensor(0.045720328, shape=(), dtype=float3..."


In [32]:
jobMatch = resultsDf[resultsDf["JOB_ID"] == row["JOB_ID"]]
minEntries = jobMatch["LOWEST_DISTANCE"].argsort()[:10]

In [33]:
#slimmedActsDf[slimmedActsDf["ACTIVITY_ID"].isin(bestMatch.loc[minEntries,:]["ACTIVITY_ID"])]["ACTIVITY_CONTENT"]

In [34]:
#def activityMatcher(row) :
 #   for id in jobsDf["JOB_ID"] :
  #      bestOverallActivityMatch = np.argmin(resultsDf[row["JOB_ID"] == id]["LOWEST_DISTANCE"])
  #  return bestOverallActivityMatch 
    

In [35]:
column_names = ["JOB_ID", "ISCO", "JOB_CONTENT", "TOP_ACTIVITY_MATCHES", "TOP_ACTIVITY_IDS", "BEST_ACTIVITY_MATCH_3SENT", "BEST_ACTIVITY_ID_3SENT", "BEST_ACTIVITY_MATCH_1SENT", "BEST_ACTIVITY_ID_1SENT", "LOWEST_DISTANCE", ]

In [36]:
topResultsDf = pd.DataFrame(columns = column_names)

In [37]:
for label in ["JOB_ID", "ISCO", "JOB_CONTENT", "ORIGINAL_CONTENT"]:
    topResultsDf[label] = jobsDf[label]

In [38]:
topResultsDf["JOB_CONTENT"] = jobsDf.apply(lambda x: " ".join(x["JOB_CONTENT"]), axis=1)

In [39]:
topResultsDf["LOWEST_DISTANCE"] = resultsDf["LOWEST_DISTANCE"]
topResultsDf["AVERAGE_DISTANCE_3SENT"] = resultsDf["AVERAGE_DISTANCE_3SENT"]

In [40]:
topResultsDf

,JOB_ID,ISCO,JOB_CONTENT,TOP_ACTIVITY_MATCHES,TOP_ACTIVITY_IDS,BEST_ACTIVITY_MATCH_3SENT,BEST_ACTIVITY_ID_3SENT,BEST_ACTIVITY_MATCH_1SENT,BEST_ACTIVITY_ID_1SENT,LOWEST_DISTANCE,ORIGINAL_CONTENT,AVERAGE_DISTANCE_3SENT
0,12220025,21420,We are a successful and independent engineerin...,NaN,NaN,NaN,NaN,NaN,NaN,0.729741,<p>Wir sind ein erfolgreiches und unabhängiges...,0.766862
1,10814567,21420,We are a renowned and successful engineering c...,NaN,NaN,NaN,NaN,NaN,NaN,0.721484,Wir sind eine renommierte und erfolgreiche Ing...,0.859877
2,12401674,21420,Bill Weyermann Partner AG has been providing e...,NaN,NaN,NaN,NaN,NaN,NaN,0.807079,Bill Weyermann Partner AG erbringt seit über\n...,0.826080
3,12473085,62100,Administration communale de ValdeRuzForstwarti...,NaN,NaN,NaN,NaN,NaN,NaN,0.866480,Administration communale de Val-de-RuzForstwar...,0.895573
4,12905109,26220;24210,The Straumann Group is internationally success...,NaN,NaN,NaN,NaN,NaN,NaN,0.837572,Le groupe Straumann connaît un succès internat...,0.866659
...,...,...,...,...,...,...,...,...,...,...,...,...
95,13469747,62100,Commune de Lausanne Service des parcs et domai...,NaN,NaN,NaN,NaN,NaN,NaN,0.785710,Commune de Lausanne - Service des parcs et dom...,0.810360
96,13494906,24210,Quality Assurance Engineer Bern Switzerland Yo...,NaN,NaN,NaN,NaN,NaN,NaN,0.841356,"Quality Assurance Engineer\n\n\nBern, Switzerl...",0.875942
97,13517245,21420;31180,Solution-oriented creative and dynamic We are ...,NaN,NaN,NaN,NaN,NaN,NaN,0.786357,"Lösungsorientiert, kreativ und dynamisch: Wir ...",0.844407
98,13461980,21620,Green BlueGarden garden and landscaping EFZLau...,NaN,NaN,NaN,NaN,NaN,NaN,0.851093,Grün & BlauGärtner/in (Garten- und Landschafts...,0.889590


In [41]:
bestNMatchReturn(toprow, resultsDf, slimmedActsDf, "AVERAGE_DISTANCE_3SENT", "ACTIVITY_ID", 10)

NameError: name 'toprow' is not defined

In [ ]:
bestMatchReturn(toprow, resultsDf, slimmedActsDf, "AVERAGE_DISTANCE_3SENT", "ACTIVITY_ID")

In [ ]:
topResultsDf["BEST_ACTIVITY_MATCH_1SENT"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "LOWEST_DISTANCE", "ACTIVITY_CONTENT"), axis=1)
topResultsDf["BEST_ACTIVITY_MATCH_3SENT"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "AVERAGE_DISTANCE_3SENT", "ACTIVITY_CONTENT"), axis=1)
topResultsDf["BEST_ACTIVITY_ID_1SENT"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "LOWEST_DISTANCE", "ACTIVITY_ID"), axis=1)
topResultsDf["BEST_ACTIVITY_ID_3SENT"] = topResultsDf.apply(bestMatchReturn, args=(resultsDf, slimmedActsDf, "AVERAGE_DISTANCE_3SENT", "ACTIVITY_ID"), axis=1)


In [ ]:
topResultsDf["TOP_ACTIVITY_IDS"] = topResultsDf.apply(bestNMatchReturn, args=(resultsDf, slimmedActsDf, "AVERAGE_DISTANCE_3SENT", "ACTIVITY_ID", 10), axis=1)
topResultsDf["TOP_ACTIVITY_MATCHES"] = topResultsDf.apply(bestNMatchReturn, args=(resultsDf, slimmedActsDf, "AVERAGE_DISTANCE_3SENT", "ACTIVITY_CONTENT", 10), axis=1)
topResultsDf

In [ ]:
annotatedJobs200 = pd.read_csv("Data/AnnotatedJobs200.csv", sep=";")

In [ ]:
topResultsDf["GREEN_IDENTITY"] = annotatedJobs200["GREEN_IDENTITY"].astype('bool')

In [ ]:
topResultsDf = topResultsDf.sort_values(axis=0, by="AVERAGE_DISTANCE_3SENT")

In [ ]:
topResultsDf

In [ ]:
topResultsDf.to_csv("PrelimActsResults200V26.csv")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(topResultsDf["AVERAGE_DISTANCE_3SENT"], density=True, label="Best match", alpha=0.5)
plt.hist(resultsDf["AVERAGE_DISTANCE_3SENT"], density=True, label="all activities", alpha=0.5)
plt.legend()

In [ ]:
plt.hist(topResultsDf["AVERAGE_DISTANCE_3SENT"][topResultsDf["GREEN_IDENTITY"] == 1.0], label="Green jobs", density=True, alpha=0.5)
plt.hist(topResultsDf["AVERAGE_DISTANCE_3SENT"][topResultsDf["GREEN_IDENTITY"] == 0.], label="not green", density=True, alpha=0.5)
plt.legend()

In [ ]:
topResultsDf[topResultsDf["AVERAGE_DISTANCE_3SENT"] < 0.85].to_csv("PrelimResultsActs200V26_GoodMatches.csv")